#### 01 - Ingest Azure Compute Documentation into databricks Unity Catalog

This notebook downloads Azure Compute documentation from GitHub, cleans Markdown content, and writes the data into a Unity Catalog–managed Delta table.

**Execution environment**
- Run this notebook on **Azure Databricks (Premium tier)** with **Unity Catalog enabled**
- Uses a **single-user Databricks cluster** (DBR 15+)
- Writes data as **Unity Catalog–managed Delta tables**

**What this notebook does**
- Downloads the latest Azure Compute documentation from the public GitHub repository  
  `MicrosoftDocs/azure-compute-docs` using a shallow Git clone
- Parses and cleans Markdown files under the `articles/` directory
- Extracts metadata such as document ID, category, title, source URL, and ingestion time
- Persists the processed documents into a governed Delta table: databricks_rag_demo.default.raw_azure_compute_docs

This notebook establishes the **raw document ingestion layer** for a Retrieval-Augmented Generation (RAG) pipeline and intentionally avoids legacy DBFS-based storage in favor of **Unity Catalog–managed data objects**.


In [0]:
%sql

----- workspace will create a default catelog with same name as the workspace, we will mostly work in this catelog
SHOW CATALOGS;

catalog
databricks_rag_demo
samples
system


In [0]:
%sql SHOW SCHEMAS IN databricks_rag_demo;

databaseName
default
information_schema


In [0]:
%run ./00_constants

In [0]:
import os
import re
from pathlib import Path
from pyspark.sql import Row
from datetime import datetime
import shutil
import stat
from collections import defaultdict

In [0]:
TARGET_ARTICLE_PATH = f"{DOC_LOCAL_DOWNLOAD_DIR}/articles"

In [0]:
def download_azure_compute_docs():

    ## delete existing download
    if os.path.exists(DOC_LOCAL_DOWNLOAD_DIR):
        print(f"Deleting {DOC_LOCAL_DOWNLOAD_DIR}")
        shutil.rmtree(DOC_LOCAL_DOWNLOAD_DIR)
    else:
        print(f"Folder does not exist: {DOC_LOCAL_DOWNLOAD_DIR}")

    try:
        import git
    except ImportError:
        # gitpython is a Python wrapper around the git command. We will use this to do git clone
        %pip install gitpython
        dbutils.library.restartPython()
    
    from git import Repo

    # SHALLOW clone
    Repo.clone_from(
        DOC_REPO_URL,
        DOC_LOCAL_DOWNLOAD_DIR,
        depth=1 # Depth = how much git history you download
    )

download_azure_compute_docs()
os.listdir(f"{TARGET_ARTICLE_PATH}")

Deleting /tmp/azure_compute_docs


['virtual-machines',
 'service-fabric',
 'container-instances',
 'virtual-machine-scale-sets',
 'azure-impact-reporting']

In [0]:
# Function to clean markdown text
def clean_markdown(md_text: str) -> str:
    # Remove code blocks
    #md_text = re.sub(r"```.*?```", "", md_text, flags=re.S)
    # Remove images
    md_text = re.sub(r"!\[.*?\]\(.*?\)", "", md_text)
    # Remove links but keep text
    md_text = re.sub(r"\[(.*?)\]\(.*?\)", r"\1", md_text)
    # Remove headings symbols
    md_text = re.sub(r"#+ ", "", md_text)
    return md_text.strip()

In [0]:
def prepare_spark_row_data():
    article_path = Path(TARGET_ARTICLE_PATH)
    rows = []
    min_length = 500  # skip stubs / TOCs

    counts = defaultdict(int)

    for md_file in article_path.rglob("*.md"):
        try:
            with open(md_file, "r", encoding="utf-8", errors="ignore") as f:
                raw_md = f.read()

            cleaned = clean_markdown(raw_md)
            if len(cleaned) < min_length:
                continue

            rel_path = str(md_file.relative_to(article_path))
            category = rel_path.split("/", 1)[0] if "/" in rel_path else rel_path

            #### Limit the docs used for each category, you can comment out if want to use all docs
            if counts[category] >= MAX_DOCS_PER_CATEGORY:
                continue

            rows.append(Row(
                doc_id=rel_path,
                source="azure-compute-docs",
                category=category,  # e.g. virtual-machines
                title=md_file.stem,
                raw_text=cleaned,
                url=f"https://learn.microsoft.com/en-us/azure/{md_file.relative_to(TARGET_ARTICLE_PATH)}",
                ingest_time=datetime.utcnow()
            ))

            counts[category] += 1

        except Exception as e:
            # Fail-safe: skip bad files
            continue
    return rows

rows = prepare_spark_row_data()
print(f"Found {len(rows)} docs")
docs_df = spark.createDataFrame(rows)

display(docs_df.limit(2))



/home/spark-7e5c2299-9b92-4577-ac1c-68/.ipykernel/2352/command-8004489376597638-3839579437:31: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ingest_time=datetime.utcnow()


Found 214 docs


doc_id source category title raw_text url ingest_time virtual-machines/hc-series-performance.md azure-compute-docs virtual-machines hc-series-performance ---
title: HC-series VM size performance
description: Learn about performance testing results for HC-series VM sizes in Azure.
ms.service: azure-virtual-machines
ms.subservice: hpc
ms.topic: concept-article
ms.date: 07/25/2024
ms.reviewer: cynthn
ms.author: padmalathas
author: cynthn
Customer intent: "As a cloud architect, I want to analyze the performance results of HC-series VM sizes, so that I can select the optimal configuration for my high-performance computing workloads."
---

HC-series virtual machine sizes

**Applies to:** :heavy_check_mark: Linux VMs :heavy_check_mark: Windows VMs :heavy_check_mark: Flexible scale sets :heavy_check_mark: Uniform scale sets

Several performance tests have been run on HC-series sizes. The following are some of the results of this performance testing.

| Workload | HB |
|-------------------------------------------------|-----------------------|
| STREAM Triad | 190 GB/s (Intel MLC AVX-512) |
| High-Performance Linpack (HPL) | 3520 GigaFLOPS (Rpeak), 2970 GigaFLOPS (Rmax) |
| RDMA latency & bandwidth | 1.05 microseconds, 96.8 Gb/s |
| FIO on local NVMe SSD | 1.3 GB/s reads, 900 MB/s writes | 
| IOR on 4 Azure Premium SSD (P30 Managed Disks, RAID0)** | 780 MB/s reads, 780 MB/writes |

MPI latency

MPI latency test from the OSU microbenchmark suite is run. Sample scripts are on GitHub

```bash
./bin/mpirun_rsh -np 2 -hostfile ~/hostfile MV2_CPU_MAPPING=[INSERT CORE #] ./osu_latency 
```



MPI bandwidth

MPI bandwidth test from the OSU microbenchmark suite is run. Sample scripts are on GitHub

```bash
./mvapich2-2.3.install/bin/mpirun_rsh -np 2 -hostfile ~/hostfile MV2_CPU_MAPPING=[INSERT CORE #] ./mvapich2-2.3/osu_benchmarks/mpi/pt2pt/osu_bw
```




Mellanox Perftest

The Mellanox Perftest package has many InfiniBand tests such as latency (ib_send_lat) and bandwidth (ib_send_bw). An example command is below.

```console
numactl --physcpubind=[INSERT CORE #] ib_send_lat -a
```

Next steps

- Read about the latest announcements, HPC workload examples, and performance results at the Azure Compute Tech Community Blogs.
- For a higher-level architectural view of running HPC workloads, see High Performance Computing (HPC) on Azure. https://learn.microsoft.com/en-us/azure/virtual-machines/hc-series-performance.md 2026-01-11T23:01:09.432523Z virtual-machines/premium-storage-performance.md azure-compute-docs virtual-machines premium-storage-performance ---
title: 'Azure premium storage: Design for high performance'
description: Design high-performance apps by using Azure premium SSD managed disks. Azure premium storage offers high-performance, low-latency disk support for I/O-intensive workloads running on Azure VMs.
author: roygara
ms.service: azure-disk-storage
ms.custom: linux-related-content
ms.topic: concept-article
ms.date: 06/29/2021
ms.author: rogarana
Customer intent: As a developer, I want to optimize application performance on premium storage, so that I can ensure my high-performance apps meet the demands of I/O-intensive workloads efficiently.
---

Azure premium storage: Design for high performance

**Applies to:** :heavy_check_mark: Linux VMs :heavy_check_mark: Windows VMs :heavy_check_mark: Flexible scale sets :heavy_check_mark: Uniform scale sets

This article provides guidelines for building high-performance applications by using Azure premium storage. You can use the instructions provided in this document combined with performance best practices applicable to technologies used by your application. To illustrate the guidelines, we use SQL Server running on premium storage as an example throughout this document.

While we address performance scenarios for the storage layer in this article, you need to optimize the application layer. For example, if you're hosting a SharePoint Farm on premium storage, you can use the SQL Server examp

In [0]:
# In Databricks notebooks, spark is: a pre-initialized SparkSession object that Databricks creates for you.

type(spark)

pyspark.sql.connect.session.SparkSession

In [0]:
# write to table

(
    docs_df
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable(RAW_DOCS_TABLE)
)

In [0]:
## Validate data is stored into the table successfully

spark.sql(f"""
    SELECT COUNT(*) FROM {RAW_DOCS_TABLE}
""").display()

count(1)
214


In [0]:
## delete this table if needed, this will clean up the environment

# spark.sql(f"""
#     DROP TABLE {RAW_DOCS_TABLE}
# """).display()